<a href="https://colab.research.google.com/github/yf591/AITuber-Projects/blob/main/youtube_chat_llm_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに
このプロジェクトでは、YouTube Liveチャットを通じて視聴者とリアルタイムで対話できるAI VTuberを作成することができます。応答生成にはファインチューニングされた大規模言語モデル（LLM）、音声合成にはVOICEVOXを活用しています。

`youtube_chat_llm_tts.ipynb` ノートブックには、以下の動作を行うAI VTuberのコードが含まれています。


## 概要
1. YouTube Data APIを使用して、指定されたYouTube Live配信からライブチャットメッセージを取得します。
2. 感情分析モデル（現在は `cardiffnlp/twitter-roberta-base-sentiment-latest` を使用）を用いて、各チャットメッセージの感情を分析します。
3. チャットメッセージとその感情に基づいて、ファインチューニングされたLLMを使用して応答を生成します。
    -   現在の実装では、ファインチューニングされた [Llama-3.1-Swallow-8B-Instruct-v0.1](https://huggingface.co/tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.1) と、このモデルをベースに私が作成したカスタムqLoraモデルを使用しています。
4. 生成された応答をVOICEVOXを使用して音声に変換します。
    -   将来のアップデートでは、カスタムのファインチューニングされた音声モデルの使用が含まれる可能性があります。
5. 生成された音声をVTuberの声として再生します。

# 目次

1. はじめに
2. 環境セットアップ
    -   2.1 ライブラリのインストール
    -   2.2 Googleドライブのマウント
    -   2.3 VOICEVOX環境構築
3. APIキーとIDの設定
4. YouTube Data APIの初期化
5. LLMの初期化
    -   5.1 量子化設定
    -   5.2 ベースモデルとトークナイザーのロード
    -   5.3 ファインチューニングモデルのロード
6. 感情分析モデルの初期化
7. 音声出力フォルダの設定
8. VOICEVOXによるテキスト音声変換
9. 関数定義
    -   9.1 ライブチャットID取得関数
    -   9.2 チャットメッセージ取得関数
    -   9.3 感情分析関数
    -   9.4 応答生成関数
    -   9.5 チャットメッセージ処理関数
    -   9.6 チャットループ関数
10. メイン関数
11. 実行

## 環境セットアップ

In [ ]:
!nvidia-smi

In [ ]:
# Googleドライブにマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/AITuber_Projects
%ls

In [ ]:
from google.colab import output

# --- モジュールのインストールとインポート ---
# Google Colab 環境で必要なライブラリをインストール
!pip install google-api-python-client # Google API クライアントライブラリ
!pip install requests # Web リクエストライブラリ
!pip install transformers # Hugging Face の Transformers ライブラリ
!pip install torch # PyTorch ライブラリ
!pip install sounddevice # 音声再生ライブラリ
!pip install scipy # 科学計算ライブラリ
!pip install soundfile # 音声ファイル操作ライブラリ
!pip install pydub # 音声ファイル操作ライブラリ
!pip install fugashi # 日本語トークナイザー
!pip install ipadic # 日本語辞書
!pip install accelerate # PyTorchの分散学習ライブラリ
!pip install bitsandbytes # 量子化ライブラリ
!pip install peft
!pip install huggingface_hub

# PortAudioライブラリをインストール
!sudo apt-get install portaudio19-dev

# sounddeviceを再インストール
!pip install --force-reinstall sounddevice

# 音声出力設定
from IPython.display import Audio, Javascript, display
def init_audio():
  display(Javascript("""
    if (!window.audio_context) {
      window.audio_context = new (window.AudioContext || window.webkitAudioContext)();
    }
  """))
init_audio()

output.clear()

In [ ]:
# # --- VOICEVOX 環境構築 ---

# VOCIVOXコアのPythonバインディングセットアップ
# !wget https://github.com/VOICEVOX/voicevox_core/releases/download/0.14.3/voicevox_core-0.14.3+cpu-cp38-abi3-linux_x86_64.whl # 一度のみ実行
!pip install voicevox_core-0.14.3+cpu-cp38-abi3-linux_x86_64.whl

# # ONNX Runtimeのダウンロード
# !wget https://github.com/microsoft/onnxruntime/releases/download/v1.13.1/onnxruntime-linux-x64-1.13.1.tgz # 一度のみ実行
# !tar xvzf onnxruntime-linux-x64-1.13.1.tgz # 一度のみ実行
# !mv onnxruntime-linux-x64-1.13.1/lib/libonnxruntime.so.1.13.1 ./ # 一度のみ実行

# # Open Jtalkの辞書ファイルダウンロード #
# !wget http://downloads.sourceforge.net/open-jtalk/open_jtalk_dic_utf_8-1.11.tar.gz  # 一度のみ実行
# !tar xvzf open_jtalk_dic_utf_8-1.11.tar.gz  # 一度のみ実行

!pip install playsound==1.3.0 # playsoundライブラリをバージョン1.3.0でインストール

In [ ]:
# VOICEVOX 用
from pathlib import Path
import voicevox_core
from voicevox_core import AccelerationMode, AudioQuery, VoicevoxCore
from playsound import playsound
# --- ここまで VOICEVOX 環境構築 ---

In [ ]:
# ライブラリのインポート
import os
import glob
import time
import datetime
import io
from googleapiclient.discovery import build # Google APIクライアントライブラリからbuildをインポート
import time
import torch # PyTorchライブラリ
from transformers import  AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig # Hugging Face Transformersライブラリからインポート
import sounddevice as sd # 音声再生ライブラリ
import soundfile as sf # 音声ファイル操作ライブラリ
import numpy as np
import requests # Webリクエストライブラリ
import threading # スレッドライブラリ
from pydub import AudioSegment
from peft import PeftModel

# --- 感情分析用のライブラリのインポート ---
from transformers import pipeline

In [ ]:
from huggingface_hub import login
from google.colab import userdata
# HuggingFaceログイン
login(userdata.get('HF_TOKEN')) # Colabのシークレットキーを使用

In [ ]:
# ベースLLMモデルのパスを設定
BASE_MODEL_PATH = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.1" # 修正: 元モデルのパスを変更

# 独自LLMモデル（qLora）のパスを設定
LLM_MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/AITuber_Projects/Llama3.1-SW-8B-it-v0.1_A100_1rep_qlora"

# ローカルで起動したVOICEVOXのAPIエンドポイントを設定(使用しなくなるため削除)
# TTS_ENDPOINT = "http://localhost:50021/audio_query"

## APIキーとIDの設定

In [ ]:
# --- APIキーとパスの設定 ---
# 環境変数からYouTube Data APIキーを取得
YOUTUBE_API_KEY = userdata.get("YOUTUBE_API_KEY")

# 環境変数から自分のYouTubeチャンネルIDを取得
YOUTUBE_CHANNEL_ID = userdata.get("YOUTUBE_CHANNEL_ID")

In [ ]:
# --- YouTube Data API の初期化 ---
# google colab認証用のライブラリ
from google.colab import auth

# google colab認証
auth.authenticate_user() # <- 追加

# YouTube Data API v3 を使用するためのサービスを初期化
youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)

## モデルのロード

In [ ]:
# --- LLM の初期化 ---
# 量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
# モデルの設定
# LLMのモデルをロードし、GPUに転送(量子化設定を適用)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    trust_remote_code=True,
    # token=token, # HuggingFaceにログインしておけば不要
    quantization_config=bnb_config, # 量子化
    device_map='auto',
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",
).to("cuda") # 修正: モデルをGPUに転送

# tokenizerの設定
# LLMのトークナイザーをロード(ベースモデルのトークナイザーを使う)
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_PATH,
    padding_side="right", # 修正: パディングを右側にする
    add_eos_token=True # 修正: EOSトークンを追加
)
if tokenizer.pad_token_id is None: # 修正: パディングトークンがない場合、EOSトークンを設定
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
# ファインチューニングモデルのロード
model = PeftModel.from_pretrained(base_model, LLM_MODEL_PATH)

In [ ]:
# --- 感情分析モデルの初期化 ---
# 日本語感情分析モデルのロード
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest") # 修正: cardiffnlp/twitter-roberta-base-sentiment-latestを使用

## VOICEVOXによるテキスト音声変換

In [ ]:
# 音声アウトプットフォルダのパス
OUTPUT_FOLDER = "/content/drive/MyDrive/Colab Notebooks/AITuber_Projects/Audio_Output"

# フォルダが存在しない場合は作成
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# 処理済みメッセージを記録するセット
processed_messages = set()

In [ ]:
# --- VOICEVOX によるテキストから音声変換 ---
def VOICEVOX(text, out='output.wav', SPEAKER_ID=47):
    open_jtalk_dict_dir = './open_jtalk_dic_utf_8-1.11' # Open JTalkの辞書ディレクトリを指定
    acceleration_mode = AccelerationMode.AUTO # 高速化モードを自動に設定

    # VOICEVOX Coreのインスタンスを作成
    core = VoicevoxCore(
        acceleration_mode=acceleration_mode, open_jtalk_dict_dir=open_jtalk_dict_dir
    )
    core.load_model(SPEAKER_ID) # 指定した話者IDのモデルを読み込み
    audio_query = core.audio_query(text, SPEAKER_ID) # テキストを音声クエリに変換
    wav = core.synthesis(audio_query, SPEAKER_ID) # 音声クエリを音声データに変換
    out_byte = Path(out) # 出力ファイルパスをPathオブジェクトに変換
    out_byte.write_bytes(wav) # 音声データをファイルに書き込み

    # pydubを使って音声の長さを取得
    sound = AudioSegment.from_file(out, format="wav")
    duration_seconds = len(sound) / 1000.0  # ミリ秒を秒に変換

    return out, duration_seconds # 出力ファイル名と再生時間を返す

In [ ]:
from IPython.display import Audio

# テスト用のテキスト (適当な文章)
test_text = "これはテスト用の文章です。ちゃんと聞こえるかな？"

# VOICEVOXで音声を生成 (SPEAKER_IDは環境に合わせて変更してください)
output_wav, _ = VOICEVOX(test_text, out='test.wav', SPEAKER_ID=3)

# 音声を再生
Audio(output_wav, autoplay=True)

### 話者一覧（参考）

| 話者        | ノーマル | あまあま | ツンツン | セクシー | ささやき | ヒソヒソ | その他              |
| :---------- | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: | :------------------ |
| 四国めたん   |    2    |    0    |    6    |    4    |   36    |   37    |                    |
| ずんだもん   |    3    |    1    |    7    |    5    |   22    |   38    | ヘロヘロ: 75<br>なみだめ: 76 |
| 春日部つむぎ |    8    |         |         |         |         |         |                    |
| 雨晴はう     |   10    |         |         |         |         |         |                    |
| 波音リツ     |    9    |         |         |         |         |         | クイーン: 65       |
| 玄野武宏     |   11    |         |         |         |         |         | 喜び: 39<br>ツンギレ: 40<br>悲しみ: 41 |
| 白上虎太郎   |   12    |         |         |         |         |         | わーい: 32<br>びくびく: 33<br>おこ: 34<br>びえーん: 35 |
| 青山龍星     |   13    |         |         |         |         |   86    | 熱血: 81<br>不機嫌: 82<br>喜び: 83<br>しっとり: 84<br>かなしみ: 85 |
| 冥鳴ひまり   |   14    |         |         |         |         |         |                    |
| 九州そら     |   16    |   15    |   18    |   17    |   19    |         |                    |
| もち子さん   |   20    |         |         |         |         |         | セクシー/あん子: 66<br>泣き: 77<br>怒り: 78<br>喜び: 79<br>のんびり: 80 |
| 剣崎雌雄     |   21    |         |         |         |         |         |                    |
| WhiteCUL    |   23    |         |         |         |         |         | たのしい: 24<br>かなしい: 25<br>びえーん: 26 |
| 後鬼        |   27    |         |         |         |         |         | ぬいぐるみver.: 28<br>人間（怒り）ver.: 87<br>鬼ver.: 88 |
| No.7        |   29    |         |         |         |         |         | アナウンス: 30<br>読み聞かせ: 31 |
| ちび式じい   |   42    |         |         |         |         |         |                    |
| 櫻歌ミコ     |   43    |         |         |         |         |         | 第二形態: 44<br>ロリ: 45 |
| 小夜/SAYO    |   46    |         |         |         |         |         |                    |
| ナースロボ＿タイプＴ | 47 |         |         |         |         |         | 楽々: 48<br>恐怖: 49<br>内緒話: 50 |
| †聖騎士 紅桜† |   51    |         |         |         |         |         |                    |
| 雀松朱司     |   52    |         |         |         |         |         |                    |
| 麒ヶ島宗麟   |   53    |         |         |         |         |         |                    |
| 春歌ナナ     |   54    |         |         |         |         |         |                    |
| 猫使アル     |   55    |         |         |         |         |         | おちつき: 56<br>うきうき: 57 |
| 猫使ビィ     |   58    |         |         |         |         |         | おちつき: 59<br>人見知り: 60 |
| 中国うさぎ   |   61    |         |         |         |         |         | おどろき: 62<br>こわがり: 63<br>へろへろ: 64 |
| 栗田まろん   |   67    |         |         |         |         |         |                    |
| あいえるたん |   68    |         |         |         |         |         |                    |
| 満別花丸     |   69    |         |         |         |         |         | 元気: 70<br>ささやき: 71<br>ぶりっ子: 72<br>ボーイ: 73 |
| 琴詠ニア     |   74    |         |         |         |         |         |                    |
| Voidoll     |   89    |         |         |         |         |         |                    |
| ぞん子       |   90    |         |         |         |         |         | 低血圧: 91<br>覚醒: 92<br>実況風: 93 |
| 中部つるぎ   |   94    |         |         |         |         |   96    | 怒り: 95<br>おどおど: 97<br>絶望と敗北: 98 |



## 各種関数の定義

### ライブチャットID取得関数

In [ ]:
# --- ライブチャットIDを取得する関数 ---
def get_live_chat_id(youtube_video_id, youtube_data_api_key):
    params = {
        'part': 'liveStreamingDetails',
        'id': youtube_video_id,
        'key': youtube_data_api_key
    }
    response = requests.get(
        'https://youtube.googleapis.com/youtube/v3/videos', params=params)
    json_data = response.json()

    if len(json_data['items']) == 0:
        return ""

    live_chat_id = json_data['items'][0]['liveStreamingDetails']['activeLiveChatId']
    return live_chat_id

### チャットメッセージ取得関数

In [ ]:
# --- チャットメッセージを取得する関数 ---
def get_live_chat_messages(live_chat_id, api_key):
    params = {
        'liveChatId': live_chat_id,
        'part': 'id,snippet,authorDetails',
        'maxResults': 200,  # 最大200まで指定可能
        'key': api_key
    }
    response = requests.get(
        'https://youtube.googleapis.com/youtube/v3/liveChat/messages', params=params)
    return response.json()

### 感情分析関数

In [ ]:
# --- 感情分析を行う関数 ---
def analyze_sentiment(text):
    # 日本語感情分析モデルで感情を分析
    result = sentiment_analyzer(text)
    # 結果を返す
    return result[0] # 結果はリストで返ってくるので最初の要素だけを返す

### 応答生成関数

In [ ]:
# --- LLMで応答を生成する関数 ---
def generate_response(text, sentiment):
    # 感情分析の結果に基づいてプロンプトを調整
    emotion_text = f"ユーザーの感情は{sentiment['label']}です。"

    # ツンデレのキャラクター設定を追加
    character_setting = (
        "あなたはツンデレで可愛い女の子です。ユーザーをご主人様と呼び、忠実でありながらも、少し反抗的な態度を取ります。"
        "ご主人様との会話を楽しんでいますが、素直に感情を表現するのが苦手です。"
        "時々、ご主人様をからかうような発言をしますが、それは愛情表現の一つです。"
    )

    # プロンプトの作成
    prompt = (
        f"{character_setting}\n"
        f"{emotion_text} ご主人様（ユーザー）が、{text}と言いました。\n"
        f"ご主人様のコメント「{text}」を繰り返してから、それに対する返答を続けてください。\n"
    )

    # 推論の実行
    input_text = f"ユーザー: {prompt}\nシステム: "

    input_ids = tokenizer.encode(
        input_text,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.encode("<|eot_id|>", add_special_tokens=False)[0],
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

    response = outputs[0][input_ids.shape[-1]:]
    response_text = tokenizer.decode(response, skip_special_tokens=True)

    # 応答テキストから、読み上げ部分と応答部分を抽出
    reading_part = text
    response_part = response_text.replace(f"{text} ", "", 1)  # 最初の読み上げ部分を削除

    # 最終的な応答テキストを返す
    final_response_text = f"{reading_part} {response_part}"
    return final_response_text

### チャットメッセージ処理関数

In [ ]:
# --- 各チャットメッセージを処理する関数 ---
def process_chat_message(item):
    # チャットメッセージのIDとテキストを取得
    message_id = item['id']
    chat_text = item['snippet']['displayMessage']

    # チャット送信者の名前を取得
    author_name = item['authorDetails']['displayName']

    # 既に処理済みのメッセージであればスキップ
    if message_id in processed_messages:
        return

    # 処理済みメッセージとして記録
    processed_messages.add(message_id)

    # ログにメッセージと送信者名を表示
    print(f"{author_name}: {chat_text}")

    # 感情分析を実行
    sentiment = analyze_sentiment(chat_text)

    # 感情分析の結果を表示
    print(f"感情分析結果: {sentiment}")

    # LLMを使って応答を生成
    response_text = generate_response(chat_text, sentiment)

    # 生成された応答を表示
    print(f"応答: {response_text}")

    # VOICEVOXを使って応答を音声に変換 (出力先を音声アウトプットフォルダに変更)
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    output_wav = os.path.join(OUTPUT_FOLDER, f"output_{timestamp}.wav")
    output_wav, duration_seconds = VOICEVOX(response_text, out=output_wav)

    # 戻り値のサイズを表示
    print(f"VOICEVOX output size: {os.path.getsize(output_wav)}")

    # Audioオブジェクトで再生
    audio = Audio(output_wav, autoplay=True)
    display(audio)

    # 推定再生時間だけ待機
    print(f"Waiting for {duration_seconds} seconds...")
    time.sleep(duration_seconds)

    # 再生後に音声ファイルを削除
    os.remove(output_wav)
    print(f"Deleted: {output_wav}")

### チャットループ関数

In [ ]:
# --- チャットメッセージを取得し処理するループ関数 ---
def chat_loop(video_id, api_key):
    # ライブチャットIDを取得
    live_chat_id = get_live_chat_id(video_id, api_key)
    if not live_chat_id:
        print("ライブチャットIDが見つかりません。")
        return

    print(f"ライブチャットID: {live_chat_id}")
    next_page_token = None

    while True:
        try:
            params = {
                'liveChatId': live_chat_id,
                'part': 'id,snippet,authorDetails',
                'maxResults': 200,
                'key': api_key,
                'pageToken': next_page_token,
            }
            response = requests.get(
                'https://youtube.googleapis.com/youtube/v3/liveChat/messages', params=params)
            response_json = response.json()

            # 取得したメッセージを表示
            for item in response_json.get('items', []):
                process_chat_message(item)

            # 次のページを取得するためのトークンを取得
            next_page_token = response_json.get('nextPageToken')

            # ポーリング間隔を取得 (ミリ秒単位)
            polling_interval_millis = response_json.get('pollingIntervalMillis', 5000)
            # ポーリング間隔を秒単位に変換して待機
            time.sleep(polling_interval_millis / 1000)

        except Exception as e:
            print(f"エラーが発生しました: {e}")
            time.sleep(60)  # エラーが発生した場合は60秒待機

### メイン関数

In [ ]:
def main():
    # 環境変数読み込み
    YOUTUBE_DATA_API_KEY = userdata.get("YOUTUBE_API_KEY")

    video_id = "ここにYoutube LIVE IDを入力" #@param {type:"string"}
    api_key = YOUTUBE_DATA_API_KEY

    # チャットループを開始
    chat_loop(video_id, api_key)

## 実行

In [ ]:
if __name__ == "__main__":
    main()